# Example of Gaussian plume inversion with ddeq

In [ ]:
import os

import cartopy.crs as ccrs
import numpy as np
import pandas as pd
import xarray as xr

# import and setup matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 100

import ddeq

from ddeq import DATA_PATH
from ddeq.smartcarb import DOMAIN

CRS = ccrs.UTM(32)
WGS84 = ccrs.PlateCarree()

# list of point sources
sources = ddeq.misc.read_point_sources()
sources

In [ ]:
PRIORS = {
    'Janschwalde': {
        'CO2': {'Q': 1000.0, 'tau': 1e10},
        'NO2': {'Q': 1.0, 'tau': 4.0*60*60},
    },
    'Schwarze Pumpe': {
        'CO2': {'Q': 1000.0, 'tau': 1e10},
        'NO2': {'Q': 1.0, 'tau': 4.0*60*60},
    },
    'Boxberg': {
        'CO2': {'Q': 1000.0, 'tau': 1e10},
        'NO2': {'Q': 1.0, 'tau': 4.0*60*60},
    },
}

In [ ]:
q = 0.995 # default of 0.995 for non-overlapping plumes
         #          or 0.990 for testing overlapping plumes of Schwarze Pumpe and Boxberg (experimental)

filename = os.path.join(ddeq.DATA_PATH, 'Sentinel_7_CO2_2015042311_o1670_l0483.nc')

data = ddeq.smartcarb.read_level2(filename, co2_noise_scenario='medium',
                                  no2_noise_scenario='high')

data = ddeq.dplume.detect_plumes(data, sources.sel(source=['Janschwalde', 'Schwarze Pumpe', 'Boxberg']),
                                 variable='NO2', variable_std='NO2_std',
                                 filter_type='gaussian', filter_size=0.5, q=q)

ddeq.vis.visualize(data, 'NO2', gas='NO2', domain=DOMAIN, winds=None,
                   do_zoom=True, show_clouds=True);

In [ ]:
data, curves = ddeq.plume_coords.compute_plume_line_and_coords(
    data, crs=CRS, radius=25e3, plume_area='area')

data = ddeq.emissions.prepare_data(data, 'CO2')
data = ddeq.emissions.prepare_data(data, 'NO2')

time = pd.Timestamp(data.time.values)
winds = ddeq.wind.read_at_sources(time, sources, product='SMARTCARB',
                                  data_path=DATA_PATH)

In [ ]:
%%time 

# takes a few minutes
data, results = ddeq.gauss.estimate_emissions(data, winds, sources, curves, priors=PRIORS,
                                              gases=['NO2', 'CO2'], fit_decay_times=[True, False])

In [ ]:
fig = ddeq.vis.plot_gauss_result(data, results, ['Janschwalde', 'Schwarze Pumpe', 'Boxberg'],
                                 'CO2', curves, crs=CRS, vmin=-20e-3, vmax=60e-3)

In [ ]:
fig = ddeq.vis.plot_gauss_result(data, results, ['Janschwalde', 'Schwarze Pumpe', 'Boxberg'],
                                 'NO2', curves, crs=CRS, vmin=-20e-6, vmax=60e-6)

Show estimated emissions

In [ ]:
for name in ['Janschwalde', 'Schwarze Pumpe', 'Boxberg']:
    print(name)
    print('CO2', '%.2f Mt/a' % ddeq.misc.kgs_to_Mtyr(results.sel(source=name).CO2_estimated_emissions))
    print('NOx', '%.2f kt/a' % ddeq.misc.kgs_to_Mtyr(1000 * results.sel(source=name).NO2_estimated_emissions))
    print()

#Janschwalde
#CO2 52.65 Mt/a
#NOx 41.04 kt/a

#Schwarze Pumpe
#CO2 15.05 Mt/a
#NOx 8.26 kt/a

#Boxberg
#CO2 nan Mt/a
#NOx 17.24 kt/a